# Train Travel Demand Modelling using Python
## 02. Demand Modelling with 2SLS

This is the second notebook out of 2 which details the steps taken to model demand using linear regression, with specific focus on addressing endogeneity issues.


<b> Context</b>

Linear regression (LR) is one of the most popular and understandable ways to model demand. The coefficients on each variable in the fitted regression equation tell us exactly how much a change in each independent variable (X) will change the dependant variable (Y).

One of the key underlying assumptions LR relies on is exogeneity of the independent variables (X). This critical assumption states that changes in the dependent variable (Y) must be caused solely by X or some other unidentified variable. Crucially, changes in Y itself cannot cause X (reverse causation). 

Where this assumption does not hold for a variable X, X is said to be endogenous and linear regression coefficients will not accurate reflect the true change in Y for each unit of change in X.

In the context of train travel demand, price (X) will be shown to be endogenous with respect to demand for train seats (Y) which distorts a plain linear regression model and must be adjusted for. 

<b> The objectives are as follows:</b>

- Identify endogeneity issues in a linear demand model
- Correct for reverse causation using Python linearmodels 2-stage least squares regression (OLS in statsmodels, IV2SLS in linearmodels)
- Obtain a truer representation of the impact of each independant variable on train travel demand 

<b> Table of Contents </b>
- [1. Prepare Dataset](#section1)<br/>
    - [1.1 Add Dummy Variables](#section1.1)<br/>
- [2. Modelling Demand (Step-by-Step with statsmodels)](#section2)<br/>
    - [2.1 Structural Demand Regression](#section2.1)<br/>
    - [2.2 Reduced Form of Price](#section2.2)<br/>
    - [2.3 2-Stage Least Squares Regression](#section2.3)<br/>
- [3. Modelling Demand (One-shot linearmodels IV2SLS)](#section3)<br/>
    - [3.1 2-Stage Least Squares Regression Analysis (2SLS)](#section3.1)<br/>
    - [3.2 Simple OLS vs IV2SLS](#section3.2)<br/>
- [4. Confirmatory Checks ](#section4)<br/>
    - [4.1 Hausman Test of Endogeneity](#section4.1)<br/>
    - [4.2 Sargan Test of Overidentification](#section4.2)<br/>
- [5. Overall Findings](#section5)<br/>

In [1]:
# Import statistical libraries
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from linearmodels.iv import compare
from linearmodels.iv import results

# Import numerical libraries
import pandas as pd
import numpy as np
import datetime as dt

# Import plotting libraries
import seaborn as sns
from matplotlib import pyplot as plt

## 1. Dataset Recap<a id="section1"></a>

As a quick recap, the cleaned dataset consists of 197,233 rows, 1 target variable (demand or *num_seats_total*) and  of 9 potential independent variables (before dummification). All data cleanup and exploratory data analysis may be found in *01_Data_Preprocessing_and_EDA.ipynb*.

<b>Target variable, Y</b>
- *total_demand*: Natural log of number of seats bought on that purchase_date. Renamed from *num_seats_sold*.

<b>Potential Independent Variables, X</b>
- *ticket_price*: Natural log of mean ticket price paid in that particular purchase transaction. Renamed from *mean_net_ticket_price*.
- *train*: Alphabet representing a unique train identifier\*.
- *isnormcabin*: Boolean value (0/1) to identify normal vs premium cabins.
- *isreturn*: Boolean value (0/1) to identify return leg of trip vs outbound or one-way trip.
- *isoneway*: Boolean value (0/1) to identify if ticket is one-way or includes return.
- *customer_cat*: Two customer categories, A & B.
- *buy_lead_time*: How many days in advance the ticket was purchased.
- *dept_dayofweek*: Day of the week for train departure.
- *dept_monthnum*: Month of the year for train departure.



In [2]:
df_raw = pd.read_csv('clean_data.csv')
df_raw.rename(columns={'num_seats_total':'total_demand', 'mean_net_ticket_price': 'ticket_price'},inplace=True)
df_raw.head()

,total_demand,ticket_price,train,isnormcabin,isreturn,isoneway,customer_cat,buy_lead_time,dept_dayofweek,dept_monthnum
0,0.0,5.239965,A,1,0,0,A,364.0,3,10
1,0.0,5.881730,B,0,1,0,A,355.0,1,5
2,0.0,5.875977,B,0,1,0,A,349.0,7,5
3,0.0,5.982349,C,0,1,0,A,348.0,2,12
4,0.0,5.897880,B,0,1,0,A,347.0,7,5


In [3]:
df_raw.describe(include = 'all', datetime_is_numeric=True).transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
total_demand,197233.0,NaN,NaN,NaN,0.573053,0.600048,0.0,0.0,0.693147,1.098612,1.94591
ticket_price,197233.0,NaN,NaN,NaN,5.222497,0.578539,0.246054,4.688288,5.223553,5.810182,6.361407
train,197233,14,B,24897,NaN,NaN,NaN,NaN,NaN,NaN,NaN
isnormcabin,197233.0,NaN,NaN,NaN,0.609589,0.487844,0.0,0.0,1.0,1.0,1.0
isreturn,197233.0,NaN,NaN,NaN,0.482374,0.49969,0.0,0.0,0.0,1.0,1.0
isoneway,197233.0,NaN,NaN,NaN,0.11707,0.321504,0.0,0.0,0.0,0.0,1.0
customer_cat,197233,2,B,153610,NaN,NaN,NaN,NaN,NaN,NaN,NaN
buy_lead_time,197233.0,NaN,NaN,NaN,63.89519,64.790811,0.0,16.0,41.0,92.0,364.0
dept_dayofweek,197233.0,NaN,NaN,NaN,4.026426,1.988973,1.0,2.0,4.0,6.0,7.0
dept_monthnum,197233.0,NaN,NaN,NaN,6.710642,3.450747,1.0,4.0,7.0,10.0,12.0


### 1.1 Add dummy variables<a id="section1.1"></a>

Dummy variables must be created for all categorical fields.

In [4]:
df = pd.get_dummies(data=df_raw, columns=['train'], drop_first=True, prefix='train', prefix_sep='_')
df = pd.get_dummies(data=df, columns=['customer_cat'], drop_first=True, prefix='cat', prefix_sep='_')
df = pd.get_dummies(data=df, columns=['dept_monthnum'], drop_first=True, prefix='month', prefix_sep='_')
df = pd.get_dummies(data=df, columns=['dept_dayofweek'], drop_first=True, prefix='day', prefix_sep='_')

In [5]:
print('After dummies are created, there are now {} columns in total.'.format(len(df.columns)))
df.columns

After dummies are created, there are now 37 columns in total.


Index(['total_demand', 'ticket_price', 'isnormcabin', 'isreturn', 'isoneway',
       'buy_lead_time', 'train_B', 'train_C', 'train_D', 'train_E', 'train_F',
       'train_G', 'train_H', 'train_I', 'train_J', 'train_K', 'train_L',
       'train_M', 'train_N', 'cat_B', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'month_12', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6',
       'day_7'],
      dtype='object')

## 2. Modelling Demand (Step-by-Step with statsmodels)<a id="section2"></a>

In this section, each regression will be done step-by-step to demonstrate the impact of endogeneity on coefficients. This can equivalently be done with a simple function call from linearmodels (shown in Section 3).

<b> Structural Demand Equation </b>

It is hypothesized that the structural demand equation is as follow:

\begin{equation}
total demand = \beta_0 + \beta_1 ticketprice + \beta_2 isnormcabin + \beta_3 isoneway + \beta_4 train + \beta_5 customer category + \beta_6 dept month + \beta_7 dept day + u_1 (error)\quad
\\
\end{equation}

<b> Justification </b> 
- *ticket_price*: Commuters demand for train travel will reduce as price increases.
- *isnormcabin*: As shown in Notebook 01, normal cabins are significantly cheaper and communter demand for these cabins will be higher.
- *isoneway*: As shown in Notebook 01, round trip tickets (one way = 0) are slightly cheaper and commuter demand for these trips may be higher as a result. Besides price, it may also be more convenient to simply have one transaction.
- *train*: Certain routes are more popular than others, perhaps due to destination or timing.
- *customer_cat*: As shown in Notebook 01, customers in category B overwhelmingly purchase cheaper tickets. This category may hence represent some sort of loyalty or super saver program that comes with imposed conditions such as a certain level of consumption that can drive higher demand.
- *dept_dayofweek*: Each day of the week has a slightly different demand pattern depending on communter travel patterns.
- *dept_monthnum*: Similar rationale as above. For example, travel may be higher in the peak December holiday month.

<b> Endogeneity of *ticket_price*</b>

There is potential reverse causation wherein changes in *total_demand* in turn affect *ticket_price*. For example, as demand increases and seats start to sell out, the train station may raise the price. Conversely, if demand is low, the station may lower the price sell out its inventory. 

The implication is that *ticketprice* will be correlated with the unobservable error term $u_1$ and the coefficient $\beta_1$ will be biased (i.e. not reflect the true impact of changes in ticket prices on demand).

<b> Instrumental Variable Candidates </b>

The two variables left out are *isreturn* and *buy_lead_time*:
- *isreturn*: Commuters will not change their travel decisions based on whether the train is one returning to base station or not. However, capacity of each base station is limited, so this status does affect supply and hence price.
- *buy_lead_time*: Although tickets purchased well in advance are cheaper, commuters may not have that luxury to plan so far ahead and may have to book tickets at whatever point in time their plans get confirmed. In contrast, this affects supply through its impact on price.

As a result, both *isreturn* and *buy_lead_time* are potential instrumental variable candidates for addressing endogeneity of price.

### 2.1 Structural Demand Regression<a id="section2.1"></a>

A simple linear regression is first done for the demand equation to obtain a baseline. As expected, <b>all variables in the structural equation are significant</b>, with a p-value of 0.000.

Note that some categorical dummy variables have p-values higher than 0.05, but the majority of the other dummies are significant.

In [6]:
y = df['total_demand']

x = df.drop(['total_demand','isreturn','buy_lead_time'],axis=1)
x = sm.add_constant(x)

sm_ols = sm.OLS(y,x).fit()

sm_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           total_demand   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     887.8
Date:                Tue, 11 Jan 2022   Prob (F-statistic):               0.00
Time:                        19:51:34   Log-Likelihood:            -1.6508e+05
No. Observations:              197233   AIC:                         3.302e+05
Df Residuals:                  197198   BIC:                         3.306e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.4245      0.021     67.555      0.000       1.383       1.466
ticket_price    -0.2110      0.003    -63.077      0.000      -0.218      -0.204
isnormcabin     -0.0511      0.004    -13.688      0.000      -0.058      -0.044
isoneway        -0.0664      0.004    -16.709      0.000      -0.074      -0.059
train_B          0.1582      0.006     25.035      0.000       0.146       0.171
train_C          0.0877      0.007     12.264      0.000       0.074       0.102
train_D          0.0261      0.007      3.612      0.000       0.012       0.040
train_E          0.1643      0.006     25.956      0.000       0.152       0.177
train_F          0.1304      0.007     19.444      0.000       0.117       0.144
train_G          0.1219      0.006     18.832      0.000       0.109       0.135
train_H          0.1305      0.007     19.567      0.000       0.117       0.144
train_I          0.0730      0.007     10.794      0.000       0.060       0.086
train_J          0.0161      0.007      2.226      0.026       0.002       0.030
train_K         -0.0393      0.009     -4.532      0.000      -0.056      -0.022
train_L          0.0073      0.009      0.796      0.426      -0.011       0.025
train_M         -0.0937      0.008    -12.005      0.000      -0.109      -0.078
train_N         -0.1606      0.013    -12.532      0.000      -0.186      -0.135
cat_B            0.2531      0.004     70.675      0.000       0.246       0.260
month_2         -0.0343      0.007     -5.071      0.000      -0.048      -0.021
month_3          0.0491      0.006      8.174      0.000       0.037       0.061
month_4          0.0230      0.007      3.533      0.000       0.010       0.036
month_5         -0.0196      0.006     -3.174      0.002      -0.032      -0.008
month_6          0.0645      0.006     10.823      0.000       0.053       0.076
month_7         -0.0602      0.006     -9.965      0.000      -0.072      -0.048
month_8         -0.0368      0.006     -5.765      0.000      -0.049      -0.024
month_9         -0.0619      0.006     -9.980      0.000      -0.074      -0.050
month_10        -0.0334      0.006     -5.527      0.000      -0.045      -0.022
month_11         0.0571      0.006      9.610      0.000       0.045       0.069
month_12         0.2169      0.006     35.671      0.000       0.205       0.229
day_2           -0.0161      0.005     -3.360      0.001      -0.025      -0.007
day_3           -0.0067      0.005     -1.407      0.160      -0.016       0.003
day_4           -0.0166      0.005     -3.508      0.000      -0.026      -0.007
day_5           -0.0068      0.005     -1.437      0.151      -0.016       0.002
day_6           -0.0101      0.005     -2.059      0.040      -0.020      -0.000
day_7           -0.0142      0.005     -3.020      0.003      -0.023      -0.005
=====

### 2.2 Reduced Form of Price<a id="section2.2"></a>

Ticket price is then modelled by adding in instrumental variable candidates to the structural demand equation. This estimation of ticket price will be uncorrelated with the error term $u_1$ and hence, allows us to get an unbiased estimate of $\beta_1$. The equation is as follows:

\begin{equation}
ticketprice = \pi_0 + \pi_1 isnormcabin + \pi_2 isoneway + \pi_3 train + \pi_4 customer category + \pi_5 dept month + \pi_6 dept day + v_1 (error)\quad
\end{equation}

It can be seen from the regression below that <b>all variables in the reduced form equation are significant</b>, with a p-value of 0.000. This equation also models price very well, with an r-squared of 0.649.

In [7]:
y_price = df['ticket_price']
x_price = df.drop(['total_demand','ticket_price'],axis=1)
x_price = sm.add_constant(x_price)

rf_ols = sm.OLS(y_price,x_price).fit()

rf_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           ticket_price   R-squared:                       0.649
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                 1.042e+04
Date:                Tue, 11 Jan 2022   Prob (F-statistic):               0.00
Time:                        19:51:34   Log-Likelihood:                -68659.
No. Observations:              197233   AIC:                         1.374e+05
Df Residuals:                  197197   BIC:                         1.378e+05
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             5.9111      0.005   1236.432      0.000       5.902       5.920
isnormcabin      -0.5948      0.002   -319.748      0.000      -0.598      -0.591
isreturn         -0.0196      0.003     -7.679      0.000      -0.025      -0.015
isoneway          0.0936      0.002     38.160      0.000       0.089       0.098
buy_lead_time    -0.0028   1.39e-05   -200.516      0.000      -0.003      -0.003
train_B           0.0299      0.004      6.837      0.000       0.021       0.038
train_C           0.2169      0.005     45.081      0.000       0.207       0.226
train_D           0.0392      0.005      8.054      0.000       0.030       0.049
train_E          -0.0362      0.004     -9.311      0.000      -0.044      -0.029
train_F           0.0114      0.004      2.769      0.006       0.003       0.019
train_G           0.0011      0.004      0.255      0.799      -0.008       0.010
train_H          -0.0219      0.004     -5.357      0.000      -0.030      -0.014
train_I           0.0430      0.005      9.321      0.000       0.034       0.052
train_J          -0.0196      0.004     -4.418      0.000      -0.028      -0.011
train_K           0.0497      0.006      8.776      0.000       0.039       0.061
train_L           0.1740      0.006     31.023      0.000       0.163       0.185
train_M          -0.0273      0.005     -5.704      0.000      -0.037      -0.018
train_N           0.1575      0.008     19.566      0.000       0.142       0.173
cat_B            -0.2180      0.002   -101.470      0.000      -0.222      -0.214
month_2           0.0009      0.004      0.226      0.821      -0.007       0.009
month_3          -0.0773      0.004    -20.979      0.000      -0.085      -0.070
month_4          -0.0209      0.004     -5.223      0.000      -0.029      -0.013
month_5          -0.0750      0.004    -19.719      0.000      -0.082      -0.068
month_6           0.0093      0.004      2.546      0.011       0.002       0.016
month_7          -0.0867      0.004    -23.276      0.000      -0.094      -0.079
month_8          -0.0176      0.004     -4.488      0.000      -0.025      -0.010
month_9          -0.1080      0.004    -28.382      0.000      -0.115      -0.101
month_10         -0.0610      0.004    -16.462      0.000      -0.068      -0.054
month_11          0.0535      0.004     14.667      0.000       0.046       0.061
month_12          0.2510      0.004     67.053      0.000       0.244       0.258
day_2            -0.0478      0.003    -16.281      0.000      -0.054      -0.042
day_3            -0.0329      0.003    -11.303      0.000      -0.039      -0.027
day_4            -0.0326      0.003    -11.205      0.000      -0.038      -0.027
day_5             0.0435      0.003     15.071      0.000       0.038       0.049
day_6             0.0365      0.003     12.163   

### 2.3 2-Stage Least Squares Regression <a id="section2.3"></a>

Ticket price is then re-predicted using the reduced form model in section 2.2, then passed into the initial demand equation.

In [8]:
y_2sls = df['total_demand']

x_2sls = df.copy()
#x_2sls.drop(['ticket_price','total_demand'], axis = 1, inplace = True)
x_2sls['pred_price'] = rf_ols.predict(x_price)
x_2sls = x_2sls.drop(['total_demand','ticket_price','isreturn','buy_lead_time'],axis=1)

x_2sls = sm.add_constant(x_2sls)
structural_model_2sls = sm.OLS(y_2sls,x_2sls).fit()
structural_model_2sls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           total_demand   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                     813.9
Date:                Tue, 11 Jan 2022   Prob (F-statistic):               0.00
Time:                        19:51:35   Log-Likelihood:            -1.6618e+05
No. Observations:              197233   AIC:                         3.324e+05
Df Residuals:                  197198   BIC:                         3.328e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           2.1915      0.048     45.233      0.000       2.097       2.286
isnormcabin    -0.1424      0.006    -22.244      0.000      -0.155      -0.130
isoneway       -0.0505      0.004    -12.326      0.000      -0.058      -0.042
train_B         0.1516      0.006     23.820      0.000       0.139       0.164
train_C         0.1159      0.007     15.730      0.000       0.101       0.130
train_D         0.0255      0.007      3.505      0.000       0.011       0.040
train_E         0.1552      0.006     24.310      0.000       0.143       0.168
train_F         0.1275      0.007     18.906      0.000       0.114       0.141
train_G         0.1149      0.007     17.610      0.000       0.102       0.128
train_H         0.1233      0.007     18.357      0.000       0.110       0.136
train_I         0.0715      0.007     10.504      0.000       0.058       0.085
train_J         0.0145      0.007      1.994      0.046       0.000       0.029
train_K        -0.0330      0.009     -3.772      0.000      -0.050      -0.016
train_L         0.0323      0.009      3.463      0.001       0.014       0.051
train_M        -0.0954      0.008    -12.149      0.000      -0.111      -0.080
train_N        -0.1372      0.013    -10.591      0.000      -0.163      -0.112
cat_B           0.2142      0.004     50.657      0.000       0.206       0.222
month_2        -0.0341      0.007     -5.005      0.000      -0.047      -0.021
month_3         0.0429      0.006      7.086      0.000       0.031       0.055
month_4         0.0257      0.007      3.915      0.000       0.013       0.039
month_5        -0.0220      0.006     -3.540      0.000      -0.034      -0.010
month_6         0.0723      0.006     12.035      0.000       0.061       0.084
month_7        -0.0616      0.006    -10.136      0.000      -0.073      -0.050
month_8        -0.0344      0.006     -5.358      0.000      -0.047      -0.022
month_9        -0.0701      0.006    -11.222      0.000      -0.082      -0.058
month_10       -0.0386      0.006     -6.344      0.000      -0.050      -0.027
month_11        0.0622      0.006     10.399      0.000       0.050       0.074
month_12        0.2378      0.006     38.173      0.000       0.226       0.250
day_2          -0.0213      0.005     -4.418      0.000      -0.031      -0.012
day_3          -0.0104      0.005     -2.174      0.030      -0.020      -0.001
day_4          -0.0208      0.005     -4.360      0.000      -0.030      -0.011
day_5          -0.0023      0.005     -0.488      0.626      -0.012       0.007
day_6          -0.0067      0.005     -1.364      0.173      -0.016       0.003
day_7          -0.0074      0.005     -1.557      0.119      -0.017       0.002
pred_price     -0.3419      0.008    -41.898      0.000      -0.358      -0.326
===========================================

## 3. Modelling Demand (One-shot linearmodels IV2SLS)<a id="section3"></a>

The same can be achieved by passing the exogenous and endogenous variables directly into linearmodels' extremely convenient IVSLS function as per below.

### 3.1 2-Stage Least Squares Regression Analysis (2SLS) <a id="section3.1"></a>

In [9]:
y = df[['total_demand']]

exo = df.drop(['total_demand', 'ticket_price','buy_lead_time','isreturn'],axis=1)

exo = sm.add_constant(exo)

endo = df[['ticket_price']]

iv = df[['isreturn','buy_lead_time']]

sm_2sls = IV2SLS(y, exo, endo, iv).fit(
    cov_type="unadjusted"
)

print(sm_2sls)

                          IV-2SLS Estimation Summary                          
Dep. Variable:           total_demand   R-squared:                      0.1260
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1259
No. Observations:              197233   F-statistic:                 2.777e+04
Date:                Tue, Jan 11 2022   P-value (F-stat)                0.0000
Time:                        19:51:36   Distribution:                 chi2(34)
Cov. Estimator:            unadjusted                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const            2.1915     0.0484     45.314     0.0000      2.0967      2.2863
isnormcabin     -0.1424     0.0064    -22.28

### 3.2 Simple OLS vs IV2SLS <a id="section3.2"></a>

Coefficient of *ticket price*, <b>$\beta_1$, is now an unbiased estimator</b> following the IV2SLS which is critical for causal analysis and decision-making. 

There are also two further observations here:
1. <b>Change in coefficient of ticket_price</b>: Coefficient has changed from -0.211 to -0.342, suggesting that endogeneity problems would have caused significant errors in estimation with only Simple Linear Regression.
2. <b>Reduction of R-squared</b>: Given 2SLS essentially strips out unwanted ticket price correlation with error, there will be some information loss and hence reduction in goodness-of-fit.

In [10]:
# Generate Simple OLS in linearmodels
# This gives the same result as section 2.1's simple OLS in statsmodels
y = df[['total_demand']]
x = df.drop(['total_demand','buy_lead_time','isreturn'],axis=1)
x = sm.add_constant(x)

sm_ols = IV2SLS(y, x, None, None).fit(cov_type="unadjusted")

In [11]:
print(compare({"Simple OLS": sm_ols, "IV2SLS": sm_2sls}))

                   Model Comparison                   
                           Simple OLS           IV2SLS
------------------------------------------------------
Dep. Variable            total_demand     total_demand
Estimator                         OLS          IV-2SLS
No. Observations               197233           197233
Cov. Est.                  unadjusted       unadjusted
R-squared                      0.1327           0.1260
Adj. R-squared                 0.1326           0.1259
F-statistic                 3.019e+04        2.777e+04
P-value (F-stat)               0.0000           0.0000
==================     ==============   ==============
const                          1.4245           2.1915
                             (67.561)         (45.314)
ticket_price                  -0.2110          -0.3419
                            (-63.083)        (-41.972)
isnormcabin                   -0.0511          -0.1424
                            (-13.689)        (-22.283)
isoneway  

/Users/rachelsng/opt/anaconda3/lib/python3.8/site-packages/linearmodels/iv/results.py:1603: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  vals = concat(


## 4. Confirmatory Checks <a id="section4"></a>

### 4.1 Hausman Test for Endogeneity <a id="section4.1"></a>

This test confirms that all endogenous variables (here, this refers to *ticket_price*) are in fact endogenous.

As P-value ~ 0.00, we can reject the hypthothesis H0 that endogenous variables are exogenous. We can confirm that *ticket_price* is endogenous and we have correctly done 2SLS to correct this.

In [12]:
sm_2sls.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 316.3442
P-value: 0.0000
Distributed: F(1,197197)
WaldTestStatistic, id: 0x7fd3d1d65d60

### 4.2 Sargan Test for Overidentification <a id="section4.2"></a>

This test confirms that the model is not overidentified, meaning that we are not using more instrumental variables than required to estimate the endogenous variable consistently. As there are two IVs introduced (*isreturn* and *buy_lead_time*) and only 1 endogenous variable, there is potentially 1 extra IV than needed.

Here, p-value is 0.0674, which is above a 0.05 significance level, meaning we do not reject H0 = Model is not overidentified.

In [13]:
sm_2sls.wooldridge_overid

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 3.3455
P-value: 0.0674
Distributed: chi2(1)
WaldTestStatistic, id: 0x7fd32020e0a0

## 5. Overall Findings <a id="section5"></a>

Overall, after all proper adjustments have been made, this simple model allows us to quantify important and specific findings about demand for train travel.

<b>Generalisable Observations</b>
1. Each 1% increase in price, results in a -0.3% decrease in demand, roughly 1/3rd the impact 
2. One way rides have slightly lower demand (-5%) compared to round trips 
3. With the baseline as Jan, December is the most popular (24% higher demand) while September is the least popular (-7% demand)


<b>Company-Specific Observations</b>
1. Commuters in Category B have 21% higher demand compared to Category A
2. Normal cabins, despite being the most sold & cheapest, are actually -14% less popular than premium ones (all else held constant)
3. With the baseline as Train A, Train B plys the most popular route (15% higher demand), while Train N operates the least popular route (-14% demand)

Importantly, this model can be used to directly forecast demand in the future and allows the train company to plan supply of trains accordingly. 